In [1]:
# Author: Zhengxiang (Jack) Wang 
# Date: 2021-08-05
# GitHub: https://github.com/jaaack-wang 
# About: Backpropagation for Stanford CS224N- NLP with Deep Learning | Winter 2019

# Table of Contents
- [1. Casual takeaways](#1)
- [2. Derivative with regard to a weight matrix](#2)
    - [2.1 An shallow neural network example](#2-1)
    - [2.2 Let's derive](#2-2)
        - [2.2.1 With regard to W](#2-2-1)
        - [2.2.2 With regard to b](#2-2-2)
    - [2.3 Tips for deriving gradients](#2-3)
    - [2.4 Updating weights: gradient descent](#2-4)
    - [2.5 Transfer learning: using pre-trained parameters](#2-5)
- [3. Computation graph and backpropagation](#3)
    - [3.1 Overview](#3-1)
    - [3.2 An example](#3-2)
    - [3.3 Generalization of computation graph in neural networks](#3-3)
    - [3.4 Debugging](#3-4)
- [4. General tricks: regularization, nonlinearities, hyperparmeters etc.](#4)
    - [4.1 Regularization](#4-1)
    - [4.2 Vectorization](#4-2)
    - [4.3 Non-linearities: activation functions](#4-3)
    - [4.4 Parameter Initialization](#4-4)
    - [4.5 Optimizers](#4-5)
    - [4.6 Learning rate](#4-6)
- [5. References](#5)
- [6. Appendix](#6)
    - [6.1 Notations](#6-1)
    - [6.2 A quick and general solution](#6-2)
    - [6.3 Sigmoid function](#6-3)
    - [6.4 Softmax function](#6-4)

<a name='1'></a>
# 1. Casual takeaways

- The year of 2019, deep learning is still kind of a craft. True even for 2021 I guess! AutoML is not the ultimate solution to learning ML.  


- Backpropagation does not always work perfectly. Mathemtically perect gradients are not guaranteed to lead to a flawless nerual netwrok. It is thus very Important to check and debug (e.g., shape convention, local gradient check when necessary)


- Why learn backpropagation when modern DL frameworks can do all these for us? Manning recommends this aritlce [Yes you should understand backprop](https://karpathy.medium.com/yes-you-should-understand-backprop-e2f06eab496b). My personal experience and feeling is that if we do not undertand the basic building blocks of a neural network, we cannot use it expertly nor can we have a say about why it works and why it does not. Think from first principles, but please learn from first principles!


- In traditional statistics, it is discouraged that we have more parameters than the training examples in a model. A rule-of-thumb back then was: parameters: training examples = 1: 10. However, this is not true with deep learning models, in which parameters: training examples = 10: 1 is not uncommon. But the secret of success behind is we regularize our model. 


- In 80s, 90s neural nets, sigmoid function is everywhere as the activation function. Just in recent time, sigmoid function  has lost popularity as the activation function, but still widely used as the activation function in the output layer for binary classification problem. 


<a name='2'></a>
# 2. Derivative with regard to a weight matrix

<a name='2-1'></a>
## 2.1 An shallow neural network example


- This example comes from lecture 3 and it is about using shallow neural network to do Named Entity Task (see lecture 3 notes). Concretely, this basic idea of this model is to predict whether a given window size of tokens have the target entity (e.g., location) as the center word, which is similar to the Skip-gram model with negative sampling. 


- This model only has three layers (or two layers, depending on whether to count the input layer as the first layer). In the figure, from bottom up, the layers are: input layer, hidden layer, and output layer. 


- In the figure, each word is represented as a vector (4 dots), so $\mathbf{x}$ is actually a matrix. For simiplicity, let's treat it as a column vector of 5 or m rows. 


- In the hidden layer, $f$ is the activation function, **but its form is not given**. Please note that: 1. $f$ takes $\mathbf{Wx + b}$ as inputs instead of just $\mathbf{x}$. 2. $\mathbf{h}$ is the output of $f$. 3. $\mathbf{h}$ is also a matrix, but for simiplicity, let's treat it as a column vector of 2 or n rows.


- $\mathbf{u}^T \mathbf{h}$ is the activitaion funtion for the output layer which takes $\mathbf{h}$, from the previous hidden layer, as inputs.


- <font color='red'>**Manning is actually not doing backpropagation here in the sense of minimizing the loss function and updating weights. He is just showing students how to apply chain rules backward in a neural network setting and how to get the shape of the derived weights right.**</font> In other words, if we really do what he showed, we will never be able to update the model's parameters. What we will get instead will just be the preset parameters themselves. 


<img src='../images/4-example-drive-gradients.png' width='600' height='300'>




<a name='2-2'></a>
## 2.2 Let's derive

<a name='2-2-1'></a>
### 2.2.1 With regard to W

- The connection between $s$ and $\mathbf{W}$ is indirect, so we need to apply chain rules as shown in the last figure: $\frac{\partial s}{\partial \mathbf{W}} = \frac{\partial s}{\partial \mathbf{h}} \frac{\partial \mathbf{h}}{\partial \mathbf{z}} \frac{\partial z}{\partial \mathbf{W}}$.  


- $\mathbf{\delta}$ (read "delta") here is equivalent to $\frac{\partial s}{\partial \mathbf{h}} \frac{\partial \mathbf{h}}{\partial \mathbf{z}} = \mathbf{u}^T f'(\mathbf{z})$. As the exact form of $f$ is unknown, we use $f'(\mathbf{z})$ to denote $\frac{\partial \mathbf{h}}{\partial \mathbf{z}}$. Manning called it the error signal, which is not accurate, because $s$ is not the loss function for the model. However, the basic logic of denoting $\frac{\partial \mathbf{h}}{\partial \mathbf{z}}$ is same, as we will reuse this term to compute gradients with regard to other variables (i.e., $\mathbf{b}$).


- Mathematically, the shape of $\mathbf{\delta}$ is a (1, n) row vector. This is because $\mathbf{u}^T$ is $(1, n) $ dimensional, $f'(\mathbf{z})$ is (n, n) dimensional. Their product is thus (1, n) dimensional. However, this does not make sense as the shape of $\mathbf{\delta}$ should align with the shape of $\mathbf{h}$, which is (n, 1) dimensional. So by shape convention, we will readjust $\mathbf{\delta}$ as $\mathbf{\delta}^T$. 


- Why is $f'(\mathbf{z})$ (n, n) dimensional? This is because: $\frac{\partial \mathbf{h}}{\partial \mathbf{z}} = \sum_{i=1}^{n} \sum_{j=1}^{n} \frac{\partial {h_i}}{\partial {z_j}}$. The result is actually a (n, n) diganoal matrix as when $i \neq j$, $\frac{\partial {h_i}}{\partial {z_j}} = 0$. 


- The result of $\frac{\partial \mathbf{z}}{\partial \mathbf{W}} = \frac{\partial}{\partial \mathbf{W}} \mathbf{W}\mathbf{x} + \mathbf{b}$ is $\mathbf{x}^T$, a row vector of m columns. (Remember $\mathbf{x}$ is treated here as a column vector of m rows for simplicity). The proof is given the in figure below. 


- Taken together, $\frac{\partial s}{\partial \mathbf{W}} = \mathbf{u}^T f'(\mathbf{z}) \mathbf{x}^T = \mathbf{\delta}^T  \mathbf{x}^T$. As $\mathbf{\delta}^T$ is (n, 1) dimensional and $\mathbf{x}^T$ is (1, m) dimensional, the result should be (n, m) dimensional, which is of the same shape with the matrix $W$. Great!


<img src='../images/4-exmp-wrt-W.png' width='600' height='300'>


<img src='../images/4-exmp-wrt-W2.png' width='600' height='300'>




<a name='2-2-2'></a>
### 2.2.2 With regard to b

- As $\frac{\partial \mathbf{z}}{\partial \mathbf{b}} = \frac{\partial}{\partial \mathbf{b}} \mathbf{W}\mathbf{x} + \mathbf{b} = \mathbf{I}$, a (m, m) identify matrix (the proof is similar to how we derive $\frac{\partial \mathbf{h}}{\partial \mathbf{z}}$ above, $\frac{\partial s}{\partial \mathbf{b}} = \mathbf{u}^T f'(\mathbf{z}) \mathbf{I} = \mathbf{u}^T f'(\mathbf{z}) = \mathbf{\delta}$.

- As we proove above, here we need to transpose $\mathbf{\delta}$ in order to follow the shape convention, therefore $\frac{\partial s}{\partial \mathbf{b}} = \mathbf{\delta}^T$, (n, 1) dimensional. 



<a name='2-3'></a>
## 2.3 Tips for deriving gradients


- The dimensionality of the weights between two layers is n by m where "n" is the number of units in the next layer and "m" is the number of units in the previous layer. 
- Please refer to the [Appendix](#6) for how to compute the gradients for both sigmoid function (binary classification) and softmax function (multi-class lassification).

<img src='../images/4-tips-for-gradients.png' width='600' height='300'>


<a name='2-4'></a>
## 2.4 Updating weights: gradient descent

- When we derive the gradients with regard to, say, $\mathbf{W}$, we can update $\mathbf{W}$ by subtracting it with the derived gradients multiplied by a given learning rate $\alpha$. Let's denote the loss of our nerual network model as $C$, then we can update $\mathbf{W}$ by implementing the following: 

$$\mathbf{W}_{new} = \mathbf{W}_{old} - \alpha \frac{\partial C}{\partial \mathbf{W}}$$. 


- This updating process is known as gradient descent. 


- By doing gradent descent, we reduces the averaged total losses of our model in executing given tasks, which in turn optimize the overall performance of the model. Please note that, the total losses of our model should always be **averaged**.



<a name='2-5'></a>
## 2.5 Transfer learning: using pre-trained parameters 


- Applying pre-trained parameters is also known as transfer learning. 



- Transfer learning is said to work well when the data the pre-trained is based on is much larger than the data we want to apply transfer learning to. 



- When our data is also considerably large, it is generally an good idea to fine-tune the pre-trained parameters to make them better suit with our data. 



<img src='../images/4-transfer-learning.png' width='600' height='300'>


<a name='3'></a>
# 3. Computation graph and backpropagation

<a name='3-1'></a>
## 3.1 Overview

- Forward propagation
- Please note that in the following figure, the extract form of $f$ is still not given. $f$ is often known as activitation function. 
- The "dot" and the "plus" sign follows the multiplication and addition rules of vectors and matrices. 

<img src='../images/4-computation-graph-example.png' width='600' height='300'>

<br>

- Backpropagation
- The backpropagation can go as far as it reaches $W$, the weights for the input layer. 

<img src='../images/4-computation-graph-example2.png' width='600' height='300'>



- The following is an illustration for a single node. 
- The process is: node receives an “upstream gradient”, has its "local gradient", and by chain rules passes on the “downstream gradient” to the next node so on and so forth. 
- If a node has multiple inputs, then the node will have multiple local grdients (see the second figure down below). **However, please note that, we usually do not derive gradients with regard to $\mathbf{x}$ because we do not want to change our input values**. Usually, $\mathbf{z}$ takes the form of $\mathbf{z} = \mathbf{Wx+b}$ and we derive the gradients with regard to both $\mathbf{W}$ and $\mathbf{b}$, namely, weights and biases.
- Backpropagation starts from the output layer and ends before it hits the input layer. 


<img src='../images/4-backprop-in-comp-graph.png' width='600' height='300'>

<br>

<img src='../images/4-backprop-in-comp-graph2.png' width='600' height='300'>


<a name='3-2'></a>
## 3.2 An example

- The figure below shows the forward propagation for $f(z, y, z) = (x+y)max(y,z)$ in a computation graph.
- Computation graph is an abstraction, so it does not produce any values unless the input values and the values for the hyperparameters are given. In this case, x = 1, y = 2, z = 0. 
- The example is just for illustration.

<img src='../images/4-comp-graph-exmp1.png' width='600' height='300'>

<br>

- Then, backpropagate. 
- Why $\frac{\partial b}{\partial y} = \mathbf{1} (y > z) = 1$ and $\frac{\partial b}{\partial y} = \mathbf{1} (z > y) = 0$. First, $b = max(y, z)$, so when we derive it with regard to the maximum value of $y$ and $z$, we are actually deriving $b$ with regard to itself. The result is apparently 1 or a vector or matrix of 1 depending on the type of $y$ and $z$ inputted. Therefore the more broad vectorized $\mathbf{1}$ is used first. However, as both $y$ and $z$ are scalars, their derivatives are also scalars. Moreover, as in this example $y > z$, therefore the real derivatives of $y$ and $z$ ($z$ does not appear in $b$ as it is smaller than $y$, so its derivate is 0) are 1 and 0 repsectively. 

<br>

- It is easy to understand that $\frac{\partial f}{\partial x} = 2$ through the chain rule, but why $\frac{\partial f}{\partial y} = 5$? This is because **the total derivative is the sum of its partial derivatives**. In this example, $y$ appears in two nodes ($a$ and $b$), so we have to compute $\frac{\partial f}{\partial y}$ through these two nodes. Alternatively, we can expand the $f = (x + y)y$, use product rule to compute $\frac{\partial f}{\partial y} = y \frac{\partial}{\partial y}(x+y) +  (x+y) \frac{\partial}{\partial y}x= 2 \times 1 + 3 \times 1 = 5$. The result is same. 

<img src='../images/4-comp-graph-exmp2.png' width='600' height='300'>

<br>

- Again, some tips to speed up the computation. The basic thinking is: we would like to cache some reusable results and reuse them instead of duplicate computation.

<img src='../images/4-comp-graph-exmp3.png' width='600' height='300'>



<a name='3-3'></a>
## 3.3 Generalization of computation graph in neural networks

- Please note that, what is not said here is that when we do forward propagation, we usually cache the intermediate results (e.g., $z$s, $y$s) for the later reuse in the backpropagation phase. As a result, the big $\mathcal{O}$ time complexity is the same for both forward propagation and backpropagation. However, this is at the cost of space complexity as we need to cache the intermediate results during the forward propagation. 


<img src='../images/4-comp-graph-generalize.png' width='600' height='300'>

<br>


<img src='../images/4-comp-graph-generalize2.png' width='600' height='300'>

- An example. The automatic differentiation relies on the symbolic expression of nodes in the forward propagation, but the corresponding derivative forms for different symbolic expressions are predefined and stored. Then during the backpropagation, we simply map the derivative forms  to the symbolic expressions and compute the gradients. 

<img src='../images/4-comp-graph-generalize3.png' width='600' height='300'>


<a name='3-4'></a>
## 3.4 Debugging 

- Two-sided gradient checking works better than the one-sided one. 

- Two-sided gradient checking: $f'(x) = \frac{f(x + h)-f(x-h)}{2h}$.

- One-sided gradient checking: $f'(x) = \frac{f(x + h)-f(x)}{h}$ or $f'(x) = \frac{f(x)-f(x-h)}{h}$.

<img src='../images/4-gradient-check.png' width='600' height='300'>



<a name='4'></a>
# 4. General tricks: regularization, nonlinearities, hyperparmeters etc.

I would recommend Andrew Ng's [Improving Deep Neural Networks- Hyperparameter, Tuning, Regularization and Optimization course]() as in his [Deep Learning Specialization on Coursera] for you to watch in order to gain deeper unstandings of all these tricks, which Manning talked about only breifly. 

<a name='4-1'></a>
## 4.1 Regularization

- Regularization prevents overfitting in the training set, which will help reduces the variance between the performances of model on the training set and the test set. 
- There are two major types of regularization methods, i.e., L1 and L2 regularization. The difference is that L1 regularization takes the sum of the absolute values of parameters as penalty to regulate their values whereas L2 regularization uses squares. The L1 regularization: $\lambda \sum_{k} |\theta_k|$
- Related reading: [Differences between L1 and L2 as Loss Function and Regularization](http://www.chioka.in/differences-between-l1-and-l2-as-loss-function-and-regularization/)
- Related reading: Stanford CS230 notes on [L1 and L2 Regularization](https://cs230.stanford.edu/section/4/#l1-and-l2-regularization)

<img src='../images/4-regularization.png' width='600' height='300'>



<a name='4-2'></a>
## 4.2 Vectorization

- Vectorization here also includes: matricization. 

<img src='../images/4-vectorization.png' width='600' height='300'>




<a name='4-3'></a>
## 4.3 Non-linearities: activation functions

- Why Relu takes off? Comparable effects with sigmoid/tanh, but much faster and cheaper to compute. 

- Sigmoid function has been little used as activitaion function in deep nets except when it is used for binary classification problem in the output layer. 

- Tanh is still popular. 

- Sigmoid function/tanh $\longrightarrow$ hard tanh $\longrightarrow$ Relu: simpler and simpler

<img src='../images/4-activation-func-old.png' width='600' height='300'>

<br>

<img src='../images/4-activation-func-new.png' width='600' height='300'>



<a name='4-4'></a>
## 4.4 Parameter Initialization

- You normally must initialize weights to small random values 
    - To avoid symmetries that prevent learning/specialization
- **Initialize hidden layer biases to 0** and output (or reconstruction) biases to optimal value if weights were 0 (e.g., mean target or inverse sigmoid of mean target)
- **Initialize all other weights** ~ Uniform(–r, r), with r chosen so numbers get neither too big or too small
- [Xavier initialization](https://proceedings.mlr.press/v9/glorot10a/glorot10a.pdf) has variance inversely proportional to [fan-in](https://en.wikipedia.org/wiki/Fan-in) $n_{in}$ (previous layer size) and [fan-out](https://en.wikipedia.org/wiki/Fan-out) $n_{out}$ (next layer size):

$$Var(W_i) = \frac{2}{n_{in} + n_{out}}$$

- There is also [He initialization](https://arxiv.org/pdf/1502.01852.pdf).

<a name='4-5'></a>
## 4.5 Optimizers

- SGD: Stochastic Gradient Descent. 

<img src='../images/4-optimizer.png' width='600' height='300'>



<a name='4-6'></a>
## 4.6 Learning rate



<img src='../images/4-learning-rate.png' width='600' height='300'>



<a name='5'></a>
# 5. References

- [Course website](http://web.stanford.edu/class/cs224n/index.html)

- [Lecture video](https://www.youtube.com/watch?v=yLYHDSv-288&list=PLoROMvodv4rOhcuXMZkNm7j3fVwBBY42z&index=4&t=2s) 

- [Lecture slide](http://web.stanford.edu/class/cs224n/slides/cs224n-2021-lecture02-wordvecs2.pdf)

<a name='6'></a>
# 6. Appendix: deriving gradients for sigmoid and softmax functions

<a name='6-1'></a>
## 6.1 Notations


In the lecture 3 notes, we learned that we commonly use cross-entropy loss function to compute the loss for both the sigmoid function and the softmax function. The cross-entropy loss function is given by: 



$$L(\mathbf{y, \hat y}) = - \mathbf{y \cdot \log \hat y} \tag{6-1}$$



where $\mathbf{y}$ and $\mathbf{\hat y}$ represent the observation and the prediction respectively. Moreover, $\mathbf{y}$ is a one-hot vector of $c$ columns (e.g., \[0, 0, 0, 1,..., 0\]) whereas $\mathbf{\hat y}$ is a column vector of $c$ rows (e.g., $\mathbf{\hat y}^T = [\hat y_1, \hat y_2, ..., \hat y_c]$). Their dot product is a scalar that equals $- \sum_{k=1}^{c} y_k \log \hat y_k$. Let's assume $y_j = 1$ for $j \in [1, c]$, which means the true class to predict, then clearly  $\sum_{k=1}^{c} y_k \log \hat y_k = y_j \log \hat y_j = \log \hat y_j$ because other negative classes labelled as 0 will cancel out the rest $\log \hat y_k$ altogether where $k \neq j$. 


More generally, if we have $m$ training examples or observations to predict, the averaged loss function can be given as follows:


$$L(\mathbf{y, \hat y}) = -\frac{1}{m} \sum_{i=1}^{m} \log \hat y_{ij} \tag{6-2}$$


Let's also denote $s(\mathbf{z})$ as the activation function for the output layer so $\mathbf{\hat y} = s(\mathbf{z})$ and $\hat y_{ij} = s(z_{ij})$. $s$ can either be a sigmoid function: 

$$s(\mathbf{z}) = \sum_{k=1}^{c} \frac{1}{1 + e^{-z_{ik}}} \tag{6-3}$$ 

or a softmax function: 

$$s(\mathbf{z}) = \sum_{k=1}^{c} \frac{e^{z_{ik}}}{\sum_{k=1}^{c} e^{z_{ik}}} \tag{6-4}$$

<br>

**Here, we want to derive the gradients of the loss function L with regard to z**.



<a name='6-2'></a>
## 6.2 A quick and general solution 

- As $L(\mathbf{y, \hat y})$ does not have direct connection with $\mathbf{z}$, we will apply the chain rule to derive the gradients of $L$ with regard to $mathbf{z}$: 

$$\frac{\partial L}{\partial \mathbf{z}} = \frac{\partial L}{\partial \mathbf{\hat y}} \frac{\partial \mathbf{\hat y}}{\partial \mathbf{z}} \tag{6-5}$$

- Instead of doing the computation in a vectorized way, we will do it iteratively by making a Jacobian Matrix whenever needed. Therefore, we will write $L = \sum_{k=1}^{c} y_k \log \hat y_k$, and $\mathbf{z} = [z_1, z_2, ..., z_c]$. There are a total of $c$ $z$s because the number of $\mathbf{z}$ must equal the number of $\mathbf{\hat y} = s(\mathbf{z})$. 


- Also to make things easier, we will do the graident computation for one training sample, and then generalize it to the entire training set. 

- We will split (6-5) into two parts, i.e., $\frac{\partial L}{\partial \mathbf{\hat y}}$ and $\frac{\partial \mathbf{\hat y}}{\partial \mathbf{z}}$, derive them one by one and put everything back together. 


---

**Let's derive!**

\- $\frac{\partial L}{\partial \mathbf{\hat y}} = \frac{\partial}{\partial \mathbf{\hat y}} [- \sum_{k=1}^{c} y_k \log \hat y_k] = \sum_{k=1}^{c} \frac{\partial}{\partial \hat y_k} [- \sum_{k=1}^{c} y_k \log \hat y_k]$. To put it graphically, the result is a $c$ by $c$ matrix as follows:


$$
\begin{bmatrix}
\frac{\partial}{\partial \hat y_1} [- y_1 \log \hat y_1] & 
\frac{\partial}{\partial \hat y_2} [- y_1 \log \hat y_1] & \cdots &
\frac{\partial}{\partial \hat y_c} [- y_1 \log \hat y_1] \\
\frac{\partial}{\partial \hat y_1} [- y_2 \log \hat y_2] & 
\frac{\partial}{\partial \hat y_2} [- y_2 \log \hat y_2] & \cdots &
\frac{\partial}{\partial \hat y_c} [- y_2 \log \hat y_2] \\
\vdots & \ddots & \vdots \\
\frac{\partial}{\partial \hat y_1} [- y_c \log \hat y_c] & 
\frac{\partial}{\partial \hat y_2} [- y_c \log \hat y_c] & \cdots &
\frac{\partial}{\partial \hat y_c} [- y_c \log \hat y_c]
\end{bmatrix}
$$

<br>

\- It is easy to see that $\sum_{k=1}^{c} \frac{\partial}{\partial \hat y_k} [\sum_{k=1}^{c} y_k \log \hat y_k]$ is derivable only when $\frac{\partial}{\partial \hat y_k} [- y_k \log \hat y_k] = - y_k \frac{1}{\hat y_k}$. Therefore, the matrix above is fact a diagonal matrix: 

$$
\begin{bmatrix}
- y_1 \frac{1}{\hat y_1} & 
0 & \cdots &
0 \\
0 & 
- y_2 \frac{1}{\hat y_2} & \cdots &
0 \\
\vdots & \ddots & \vdots \\
0 & 
0 & \cdots &
- y_c \frac{1}{\hat y_c}
\end{bmatrix}
$$

<br>

\- We are not done yet! As we know, $\mathbf{y}$ is a one-hot vector with only $y_j = 1$ and the rest $\sum_{k \neq j} y_k = 0$. Therefore, $- y_k \frac{1}{\hat y_k}$ will only have non-zero value when $k = j$. That means, the above matrix in fact is a "one-hot" matrix! It should looks something like the following where $j = [1, c]$:


$$
\begin{bmatrix}
0 & 
0 & \cdots &
0 \\
0 & 
- \frac{1}{\hat y_j} & \cdots &
0 \\
\vdots & \ddots & \vdots \\
0 & 
0 & \cdots &
0
\end{bmatrix}
$$

<br>

\- By the same logic, $\frac{\partial \mathbf{\hat y}}{\partial \mathbf{z}}$ can be visualized as follows (also a c by c matrix):

$$
\begin{bmatrix}
\frac{\partial y_1}{\partial \hat z_1} & 
\frac{\partial y_1}{\partial \hat z_2} & \cdots &
\frac{\partial y_1}{\partial \hat z_c} \\
\frac{\partial y_2}{\partial \hat z_1} & 
\frac{\partial y_2}{\partial \hat z_2} & \cdots &
\frac{\partial y_2}{\partial \hat z_c}  \\
\vdots & \ddots & \vdots \\
\frac{\partial y_c}{\partial \hat z_1} & 
\frac{\partial y_c}{\partial \hat z_2} & \cdots &
\frac{\partial y_c}{\partial \hat z_c} 
\end{bmatrix}
$$

<br>

\- Therefore, if we multiply the above two matrices, we will still get a $c$ by $c$ matrix, which, however, only has a non-zero value at $j$th row and $j$th column taking the form of $- \frac{1}{\hat y_j} \frac{\partial y_j}{\partial \hat z_j}$. 


<br>

---

<font color='blue'>**Answers here**</font>

**In multivariate calculus, the total derivate is the sum of all partial derivates.** That means, $\frac{\partial L}{\partial \mathbf{z}}$ simply equals $- \frac{1}{\hat y_j} \frac{\partial y_j}{\partial \hat z_j}$ as the rest partial derivates all equal 0. 


More generally, for m training examples, the averaged loss will be $-\frac{1}{m} \sum_{i=1}^{m} \frac{1}{\hat y_{ij}} \frac{\partial y_{ij}}{\partial \hat z_{ij}}$ where $j \in [1, n]$ but can be variable across the training examples. This result is actually equal to deriving gradients of loss function as in (6-2) with regard to $z_{ij}$:

$$\frac{\partial L}{\partial \mathbf{z}} = \frac{\partial}{\partial {y_{ij}}}[-\frac{1}{m} \sum_{i=1}^{m} \log \hat y_{ij}] \frac{\partial y_{ij}}{\partial {z_{ij}}} = -\frac{1}{m} \sum_{i=1}^{m} \frac{1}{\hat y_{ij}} \frac{\partial y_{ij}}{\partial \hat z_{ij}}$$


Or, in short:


$$\frac{\partial L}{\partial \mathbf{z}} = -\frac{1}{m} \sum_{i=1}^{m} \frac{1}{\hat y_{ij}} \frac{\partial y_{ij}}{\partial \hat z_{ij}} \tag{6-6}$$



<a name='6-3'></a>
## 6.3 Sigmoid function

The loss function for the sigmoid function is usaully given as: 


$$L(\mathbf{y, \hat y}) = -\frac{1}{m} \sum_{i=1}^{m} [y_{i} \log \hat y_{i} + (1-y_{i}) \log (1 - \hat y_{i} )] \tag{6-7}$$


Accordinly, $\mathbf{y}$ is a (m, 1) vector where $y_{i}$ takes values of either 0 or 1. However, in (6-2), $y_{i}$ is a one-hot vector that has $c$ dimensions, but not a scalar. (6-7) works fine for sigmoid function as it only deals with binary classification (unless we apply sigmoid function iteratively using one-versus-all approach). (6-7) may also look more intuitive for many to derive gradients. Nevertheless, for consistentcy, we will still use the notations established in [6.1](#6-1) to derive gradients with regard to $z$.

---

**Let's derive!**


\- We will simply use the formula given in (6-6): $\frac{\partial L}{\partial \mathbf{z}} = -\frac{1}{m} \sum_{i=1}^{m} \frac{1}{\hat y_{ij}} \frac{\partial y_{ij}}{\partial \hat z_{ij}}$.


\- For sigmoid function, according to (6-3), we know $y_{ij} = \frac{1}{1 + e^{-z_{ij}}}$. Therefore, $\frac{\partial y_{ij}}{\partial \hat z_{ij}}$ can be computed as follows:

$$\frac{\partial}{\partial z_{ij}} \frac{1}{1 + e^{-z_{ij}}} = \frac{\partial}{\partial z_{ij}} ({1 + e^{-z_{ij}}})^{-1} = - ({1 + e^{-z_{ij}}})^{-2} \frac{\partial}{\partial z_{ij}} e^{-z_{ij}} = - ({1 + e^{-z_{ij}}})^{-2} (- e^{-z_{ij}}) = ({1 + e^{-z_{ij}}})^{-2} e^{-z_{ij}}$$


<br>

\- Therefore, $\frac{\partial L}{\partial \mathbf{z}} =  - \frac{1}{m} \sum_{i=1}^{m} \frac{1}{({1 + e^{-z_{ij}}})^{-1}} ({1 + e^{-z_{ij}}})^{-2} e^{-z_{ij}} =  - \frac{1}{m} \sum_{i=1}^{m} \frac{1}{({1 + e^{-z_{ij}}})^{-1}} e^{-z_{ij}} = - \frac{1}{m} \sum_{i=1}^{m} \frac{ e^{-z_{ij}}}{({1 + e^{-z_{ij}}})^{-1}}$

<br>


\- A trick: $- \frac{1}{m} \sum_{i=1}^{m} \frac{ e^{-z_{ij}}}{({1 + e^{-z_{ij}}})^{-1}} = - \frac{1}{m} \sum_{i=1}^{m} [1 - \frac{1}{({1 + e^{-z_{ij}}})^{-1}}] = \frac{1}{m} \sum_{i=1}^{m}[\hat y_{ij} - y_{ij}]$.

<br>

**\- Answer:** $\frac{\partial L}{\partial \mathbf{z}} = \frac{1}{m} \sum_{i=1}^{m}[\hat y_{ij} - y_{ij}]$.






<a name='6-4'></a>
## 6.4 Softmax function

A reminder, the quotient rule in calculus: 

$$\frac{\partial}{\partial x} \frac{f(x)}{g(x)} = \frac{g(x)\frac{\partial}{\partial x}f(x) - f(x)\frac{\partial}{\partial x}g(x)}{g(x)^{2}}$$

---

**Let's derive!**


\- We will simply use the formula given in (6-6): $\frac{\partial L}{\partial \mathbf{z}} = -\frac{1}{m} \sum_{i=1}^{m} \frac{1}{\hat y_{ij}} \frac{\partial y_{ij}}{\partial \hat z_{ij}}$.


\- For sigmoid function, according to (6-4), we know $y_{ij} = \frac{e^{z_{ij}}}{\sum_{k=1}^{c} e^{z_{ik}}} $. Therefore, $\frac{\partial y_{ij}}{\partial \hat z_{ij}}$ can be computed as follows:

$$\frac{\partial}{\partial z_{ij}} \frac{e^{z_{ij}}}{\sum_{k=1}^{c} e^{z_{ik}}} = \frac{1}{(\sum_{k=1}^{c} e^{z_{ik}})^2} ((\sum_{k=1}^{c} e^{z_{ik}})\frac{\partial}{\partial z_{ij}}e^{z_{ij}} - e^{z_{ij}}  \frac{\partial}{\partial z_{ij}} \sum_{k=1}^{c} e^{z_{ik}}) = \frac{1}{(\sum_{k=1}^{c} e^{z_{ik}})^2}((\sum_{k=1}^{c} e^{z_{ik}})e^{z_{ij}}  - e^{z_{ij}} e^{z_{ij}})$$

<br>

\- Reaggrange the result above, we get:


$$\frac{1}{(\sum_{k=1}^{c} e^{z_{ik}})^2}((\sum_{k=1}^{c} e^{z_{ik}})e^{z_{ij}}  - e^{z_{ij}} e^{z_{ij}}) = \frac{e^{z_{ij}}}{\sum_{k=1}^{c} e^{z_{ik}}} - (\frac{e^{z_{ij}}}{\sum_{k=1}^{c} e^{z_{ik}}})^2 = \hat y_{ij} - \hat y_{ij}^2 = \hat y_{ij}(1-\hat y_{ij})$$


<br>

\- Therefore, $\frac{\partial L}{\partial \mathbf{z}} = -\frac{1}{m} \sum_{i=1}^{m} [\frac{1}{\hat y_{ij}} \hat y_{ij}(1-\hat y_{ij})] = \frac{1}{m} \sum_{i=1}^{m} [\hat y_{ij}-1]$


<br>

\- A trick, as $y_{ij} = 1$, the above result can also be rewritten as $\frac{1}{m} \sum_{i=1}^{m} [\hat y_{ij}-1] = \frac{1}{m} \sum_{i=1}^{m} [\hat y_{ij}-y_{ij}]$.


<br>

**\- Answer:** $\frac{\partial L}{\partial \mathbf{z}} = \frac{1}{m} \sum_{i=1}^{m} [\hat y_{ij}-y_{ij}]$.



<br>

<font color='blue'>Notes</font>
    
My solution is different than that suggested by Manning's slide as in [2.3 Tips for deriving gradients](#2-3). By his suggestion, we will derive the gradients numerically and discuss the cases when $z_{ik} = z_{ij}$ and when $z_{ik} \neq z_{ij}$. Just in case you are interested, we can do that as follows:

\- Since we have already discussed the case when $z_{ik} = z_{ij}$, let's see how to dervie $y_{ik}$ when $z_{ik} \neq z_{ij}$. It should be: 

$$\sum_{k \neq j}^{c} \frac{\partial}{\partial z_{ik}} \hat y_{ij} = \sum_{k \neq j}^{c} \frac{\partial}{\partial z_{ik}} \frac{e^{z_{ij}}}{\sum_{k=1}^{c} e^{z_{ik}}} = \sum_{k \neq j}^{c} \frac{1}{(\sum_{k=1}^{c} e^{z_{ik}})^2} ((\sum_{k=1}^{c} e^{z_{ik}})\frac{\partial}{\partial z_{ik}}e^{z_{ij}} - e^{z_{ij}}  \frac{\partial}{\partial z_{ik}} \sum_{k=1}^{c} e^{z_{ik}}) = \sum_{k \neq j}^{c} \frac{1}{(\sum_{k=1}^{c} e^{z_{ik}})^2}((\sum_{k=1}^{c} e^{z_{ik}} 0  - e^{z_{ij}} e^{z_{ik}}) = \sum_{k \neq j}^{c} \frac{- e^{z_{ij}} e^{z_{ik}}}{(\sum_{k=1}^{c} e^{z_{ik}})^2}$$

\- Rearrange:

$$ \sum_{k \neq j}^{c} \frac{- e^{z_{ij}} e^{z_{ik}}}{(\sum_{k=1}^{c} e^{z_{ik}})^2} = - \frac{e^{z_{ij}}}{\sum_{k=1}^{c} e^{z_{ik}}} \frac{e^{z_{ik}}}{\sum_{k=1}^{c} e^{z_{ik}}} = \sum_{k \neq j}^{c} [- \hat y_{ij} \hat y_{ik}] = - \hat y_{ij} \sum_{k \neq j}^{c} \hat y_{ik}$$

<br>

\- Then take everything together:

$$\frac{\partial L}{\partial \mathbf{z}} = \frac{\partial L}{\partial \mathbf{\hat y}} (\hat y_{ij}(1-\hat y_{ij}) - \hat y_{ij} \sum_{k \neq j}^{c} \hat y_{ik}) = -\frac{1}{m} \sum_{i=1}^{m} [(y_{ij} \frac{\hat y_{ij)}(1-\hat y_{ij})}{\hat y_{ij}}) - (\sum_{k \neq j}^{c} [(y_{ik} \frac{1}{\hat y_{ik}}])(\hat y_{ij} \sum_{k \neq j}^{c} \hat y_{ik})] =  -\frac{1}{m}  \sum_{i=1}^{m} [(1-\hat y_{ij}) -  \hat y_{ij} \sum_{k \neq j}^{c} [(y_{ik} \frac{1}{\hat y_{ik}}\hat y_{ik})]] = -\frac{1}{m}  \sum_{i=1}^{m} [(1-\hat y_{ij}) -  \hat y_{ij} \sum_{k \neq j}^{c} y_{ik}]$$

\- As we know, only $y_{i} = 1$, thus:


$$-\frac{1}{m}  \sum_{i=1}^{m} [(1-\hat y_{ij}) -  \hat y_{ij} \sum_{k \neq j}^{c} y_{ik}] = -\frac{1}{m}  \sum_{i=1}^{m} [(1-\hat y_{ij}) -  \hat y_{ij} 0] = -\frac{1}{m}  \sum_{i=1}^{m} [1-\hat y_{ij}] = \frac{1}{m}  \sum_{i=1}^{m} [\hat y_{ij} - 1]= \frac{1}{m}  \sum_{i=1}^{m} [\hat y_{ij} - y_{ij}]$$

\- Finally, we get the same result as above!


<br>

<font color='blue'>Notes</font>

I would recommend using the graphic solution because when doing pure numerical compuation, it is very easy to make mistakes or forget what you have already done. Pure numerical compuation can be very tricky. 